## Decision Trees

Decision trees for modeling choices have been around for a very long time, but statistical decision trees are a more recent development.

__Decision Tree Learning__ refers to the statisitcal modeling thatt uses a form of decision trees, where node splits are decided based on an information metric.

Decision trees and other tree based methods rely on the ability to split  data based on information from features, meaning that we need a mathematical definition of information and the ability to measure it.

### Terminology
> - __Splitting__, it refers to the act of taking a node and spllitting it whether its _True_ or _False_.

> - __Nodes__, they are the points where data comes in, and or goes out.
>   - __Root Node__, they are the initial condition or feature where we check or split on.
>   - __Leaf (Terminal) Nodes__, they are the end of the line for conditions or features where no data comes out.
>   - __Parent/Children Nodes__, They refer to the relationship between nodes, and how they are structured.

> - __Tree Branches (Sub Trees)__, they are the offshoots of our tree where data splits.

> - __Pruning__, its the act of shortening a tree by removing redundant data by removing a branch and transforming it into a leaf.


## Gini Impurity

It is a mathematical measure of how pure the information in a data set is. We could think of it as a measurement of class uniformity. This is given by the following equation.

$$
G(Q) = \sum_{c\in C}p_c (1-p_C)
$$

Wih $p_c$ equals to the probability of it belonging to the class $c$.

$$
p_c = \frac{1}{N_Q} \sum_{x \in Q}\mathbf{1}(y_{class}=c)
$$

If we plot out this function we can see that it is a curve starting at $(0,0)$ going up to $0.5$ and ending back at $(1,0)$. Meaning our max value for a gini impurity is $0.5$. 

If we get a Gini Impurity of 0, it means it is equals to one single class.
